<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>


# Interacting with Queues in Storage

## Azure Actions Covered

* 

In this lecture, we're going to take a look at how to interact with queues in storage containers via the Python SDK.

To begin, we import our usual libraries as well as any useful environment variables (e.g. `AZURE_SUBSCRIPTION_ID`). 

In [1]:
# For generating storage account name
import random

from azure.identity import AzureCliCredential, DefaultAzureCredential, InteractiveBrowserCredential
from azure.mgmt.resource import ResourceManagementClient
# New import
from azure.storage.queue import QueueClient, QueueServiceClient, QueueMessage
# Queue imports

from settings import AZURE_SUBSCRIPTION_ID

In [2]:
credential = AzureCliCredential()

In [3]:
queue_client = QueueClient(
    account_url='https://benbstorage1234.queue.core.windows.net',
    queue_name='bens-queue-1',
    credential=credential
)

In [7]:
queue = queue_client.create_queue()

In [20]:
queue_client.send_message("My first message")

{'id': '5b0b3330-8e67-416a-9e5a-5ba9cea9f79b', 'inserted_on': datetime.datetime(2023, 5, 15, 5, 18, 7, tzinfo=datetime.timezone.utc), 'expires_on': datetime.datetime(2023, 5, 22, 5, 18, 7, tzinfo=datetime.timezone.utc), 'dequeue_count': None, 'content': 'My first message', 'pop_receipt': 'AgAAAAMAAAAAAAAA9YBtouyG2QE=', 'next_visible_on': datetime.datetime(2023, 5, 15, 5, 18, 7, tzinfo=datetime.timezone.utc)}

In [21]:
queue_client.send_message("My second message")

{'id': '31ac7778-d5c9-4c89-81c7-fac644a4d20c', 'inserted_on': datetime.datetime(2023, 5, 15, 5, 18, 9, tzinfo=datetime.timezone.utc), 'expires_on': datetime.datetime(2023, 5, 22, 5, 18, 9, tzinfo=datetime.timezone.utc), 'dequeue_count': None, 'content': 'My second message', 'pop_receipt': 'AgAAAAMAAAAAAAAAwOQso+yG2QE=', 'next_visible_on': datetime.datetime(2023, 5, 15, 5, 18, 9, tzinfo=datetime.timezone.utc)}

In [22]:
queue_client.peek_messages(max_messages=2)

[{'id': '2f8cbc3a-ae75-406c-acb1-7a084c1fc07a', 'inserted_on': datetime.datetime(2023, 5, 15, 5, 10, 31, tzinfo=datetime.timezone.utc), 'expires_on': datetime.datetime(2023, 5, 22, 5, 10, 31, tzinfo=datetime.timezone.utc), 'dequeue_count': 1, 'content': 'My first message', 'pop_receipt': None, 'next_visible_on': None},
 {'id': '5b0b3330-8e67-416a-9e5a-5ba9cea9f79b', 'inserted_on': datetime.datetime(2023, 5, 15, 5, 18, 7, tzinfo=datetime.timezone.utc), 'expires_on': datetime.datetime(2023, 5, 22, 5, 18, 7, tzinfo=datetime.timezone.utc), 'dequeue_count': 0, 'content': 'My first message', 'pop_receipt': None, 'next_visible_on': None}]

In [23]:
first_message = queue_client.receive_message()

In [24]:
first_message

{'id': '2f8cbc3a-ae75-406c-acb1-7a084c1fc07a', 'inserted_on': datetime.datetime(2023, 5, 15, 5, 10, 31, tzinfo=datetime.timezone.utc), 'expires_on': datetime.datetime(2023, 5, 22, 5, 10, 31, tzinfo=datetime.timezone.utc), 'dequeue_count': 2, 'content': 'My first message', 'pop_receipt': 'AgAAAAMAAAAAAAAArZwWuOyG2QE=', 'next_visible_on': datetime.datetime(2023, 5, 15, 5, 18, 44, tzinfo=datetime.timezone.utc)}

In [25]:
queue_client.update_message(
    message=first_message.id,
    pop_receipt=first_message.pop_receipt,
    content="Was this my first message?"
)

{'id': '2f8cbc3a-ae75-406c-acb1-7a084c1fc07a', 'inserted_on': None, 'expires_on': None, 'dequeue_count': None, 'content': 'Was this my first message?', 'pop_receipt': 'AwAAAAMAAAAAAAAAgvLRsOyG2QECAAAA', 'next_visible_on': datetime.datetime(2023, 5, 15, 5, 18, 32, tzinfo=datetime.timezone.utc)}

In [26]:
queue_client.get_queue_properties()

{'name': 'bens-queue-1', 'metadata': {}, 'approximate_message_count': 4}

In [27]:
messages = queue_client.receive_messages()

In [28]:
for msg in messages:
    print(msg.content)

My first message
My second message
My second message
Was this my first message?


In [32]:
messages = queue_client.receive_messages()

In [33]:
for msg in messages:
    print(msg.id)
    queue_client.delete_message(msg)

5b0b3330-8e67-416a-9e5a-5ba9cea9f79b
31ac7778-d5c9-4c89-81c7-fac644a4d20c
7f166b84-da9b-4c1c-8256-56969515acb4
2f8cbc3a-ae75-406c-acb1-7a084c1fc07a


In [34]:
queue_client.delete_queue()